$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

In [1]:
!pip install pennylane > out

# Efficient Quantum Modular Arithmetics for the ISQ Era

This notebook implements the following operators:

1. **Inplace Quantum-Classic Adder**: $Add_{\text{in}}(k, N) \ket{a} \rightarrow \ket{a+k \mod N}$ 

2. **Outplace Quantum-Classic Adder**: $Add_{\text{out}}(k,N)\ket{a}\ket{0} \rightarrow \ket{a}\ket{a+k \mod N}$ 

3. **Inplace Quantum-Quantum Adder**: $Add_{\text{in}}(N)\ket{a}\ket{b} \rightarrow \ket{a}\ket{a+b \mod N}$ 

4. **Outplace Quantum-Quantum Adder**: $Add_{\text{out}}(N)\ket{a}\ket{b}\ket{0} \rightarrow \ket{a}\ket{b}\ket{a+b \mod N}$ 

5. **Outplace Quantum-Classic Multiplier**: $Mult_{\text{out}}(k, N)\ket{a}\ket{b} \rightarrow \ket{a}\ket{b + ka \mod N}$

6. **Inplace Quantum-Classic Multiplier**: $Mult_{\text{in}}(k, N)\ket{a} \rightarrow \ket{ka \mod N}$ 

7. **Outplace Quantum-Quantum Multiplier**: $Mult_{\text{out}}(N)\ket{a}\ket{b}\ket{0} \rightarrow \ket{a}\ket{b}\ket{ab \mod N}$ 

8. **Modular Exponential Operator**: $Exp(a, N)\ket{x}\ket{1}\ket{0} \rightarrow \ket{x}\ket{a^x \mod N}\ket{0}$ 


These operators are critical components in many quantum algorithms, and their efficient implementation is essential for leveraging the power of quantum computing.

In [2]:
## Libraries
import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt

In [3]:
import numpy as np

def binary_to_integer(vOutput):
    """
    Convert a binary vector 'vOutput' to an integer.
    """
    vPows = 2**np.array(range(len(vOutput)))  # Create an array of powers of 2
    vPows = vPows[::-1]  # Reverse the array to match binary order
    return np.dot(vOutput, vPows)  # Compute the dot product to convert to an integer

# Example usage
vOutput = np.array((1, 1, 0, 0))
print(binary_to_integer(vOutput))


12


## 1. *Inplace* Quantum-Classic Adder
This operator performs a modular addition of a quantum register with a classical constant.
$$Add_{\text{in}}(k, N) \ket{a} \rightarrow \ket{a+k \mod N}$$ 


![inplace_q_c_adder](https://github.com/pifparfait/Efficient-Quantum-Modular-Arithmetics/blob/main/img/add_q_c_in.png?raw=true)

In [4]:
import pennylane as qml
import numpy as np

def sum_k(k, wires):
    """
    Apply a rotation gate to each wire in 'wires', rotating by an angle 'k * pi / 2^j'.
    """
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])

def add_in_k_N(k, N, wires_a, wires_aux, is_in_standard_basis=True):
    """
    Perform in-place modular addition of 'k' to the quantum register 'wires_a' modulo 'N'.
    'wires_aux' is an auxiliary wire used for intermediate calculations.
    'is_in_standard_basis' specifies whether the input state is in the standard basis (True) or Fourier basis (False).
    """
    
    # Set the Fourier basis if 'is_in_standard_basis' is True
    if is_in_standard_basis:
        qml.QFT(wires=wires_a)  # Step 1
    
    # Step 1: Add 'k' to the register
    sum_k(k, wires_a)
    
    # Step 2: Subtract 'N'
    qml.adjoint(sum_k)(N, wires_a)
    
    # Step 3: Conditionally add 'N' back
    qml.adjoint(qml.QFT)(wires=wires_a)
    qml.CNOT(wires=[wires_a[0], wires_aux[0]])  # Use the most significant bit to control the auxiliary bit
    qml.QFT(wires=wires_a)
    
    # Step 4: Add 'N' to the register conditionally
    qml.ctrl(sum_k, control=wires_aux[0])(N, wires_a)
    
    # Step 5: Clear the auxiliary bit
    qml.adjoint(sum_k)(k, wires_a)
    
    # Step 6: Conditionally add 'N' back
    qml.adjoint(qml.QFT)(wires=wires_a)
    qml.PauliX(wires=wires_a[0])
    qml.CNOT(wires=[wires_a[0], wires_aux[0]])  # Use the most significant bit to control the auxiliary bit
    qml.PauliX(wires=wires_a[0])
    qml.QFT(wires=wires_a)
    
    # Step 7: Add 'k' to the register
    sum_k(k, wires_a)
    
    # Recover the standard basis if 'is_in_standard_basis' is True
    if is_in_standard_basis:
        qml.adjoint(qml.QFT)(wires=wires_a)

# Create the device
nWireCount = 10
dev = qml.device("default.qubit", wires=nWireCount, shots=1)

@qml.qnode(dev)
def circuit_add_in_k_N(a, k, N, wires_a, wires_aux):
    qml.BasisEmbedding(a, wires=wires_a)
    add_in_k_N(k, N, wires_a, wires_aux)
    return qml.sample()

# Set the wires
wires_a = list(range(1, 4))
wires_aux1 = 0
wires_aux2 = [4]

# Example
N = 5  # Input value between [0, 7]
a = 4  # Input value between [0, N-1]
k = 3  # Input value between [0, N-1]
wires_a = [wires_aux1] + wires_a
vOutput = circuit_add_in_k_N(a, k, N, wires_a, wires_aux2)

print(a, "+", k, "mod", N, "=", binary_to_integer(vOutput[wires_a]))


4 + 3 mod 5 = 2


## 2. *Outplace* Quantum-Classic Adder 
Similar to the previous operator but in an outplace fashion, where the result is stored in a separate quantum register.
$$Add_{\text{out}}(k,N)\ket{a}\ket{0} \rightarrow \ket{a}\ket{a+k \mod N}$$ 

![outplace_q_c_adder](https://github.com/pifparfait/Efficient-Quantum-Modular-Arithmetics/blob/main/img/add_q_c_out.png?raw=true)

In [5]:
def add_out_k_N(k, N, wires_a, wires_res, wires_aux):
    """
    Perform an outplace modular addition with a classical constant 'k' modulo 'N' on a quantum register 'wires_a'.
    The result is stored in 'wires_res'.

    :param k: Classical constant to be added modulo N.
    :param N: The modulus.
    :param wires_a: Quantum wires for input.
    :param wires_res: Quantum wires for result.
    :param wires_aux: Quantum wires for auxiliary operations.
    """
    if len(wires_aux) < 2:
        print("Not enough ancillary wires")
        return

    # Set 'a' using CNOT gates
    for i in range(len(wires_res)):
        qml.CNOT(wires=[wires_a[i], wires_res[i]])

    new_wires_res = [wires_aux[0]] + wires_res
    add_in_k_N(k, N, new_wires_res, [wires_aux[1]])

# Create the device
nWireCount = 8
dev = qml.device("default.qubit", wires=nWireCount, shots=1)

@qml.qnode(dev)
def circuit_add_out_k_N(a, k, N, wires_a, wires_res, wires_aux):
    qml.BasisEmbedding(a, wires=wires_a)
    add_out_k_N(k, N, wires_a, wires_res, wires_aux)
    return qml.sample()

# Set wires
wires_a = list(range(0, 3))
wires_res = list(range(3, 6))
wires_aux = [6, 7]

# Example
N = 7  # Input value between [0, 7]
a = 5  # Input value between [0, N-1]
k = 3  # Input value between [0, N-1]
vOutput = circuit_add_out_k_N(a, k, N, wires_a, wires_res, wires_aux)
print(a, "+", k, "mod", N, "=", binary_to_integer(vOutput[wires_res]))


5 + 3 mod 7 = 1


## 3. *Inplace* Quantum-Quantum Adder 
This operator adds two quantum registers modulo N.
$$Add_{\text{in}}(N)\ket{a}\ket{b} \rightarrow \ket{a}\ket{a+b \mod N}$$ 

![inplace_q_q_adder](https://github.com/pifparfait/Efficient-Quantum-Modular-Arithmetics/blob/main/img/add_q_q_in.png?raw=true)

In [6]:
def add_in_N(N, wires_a, wires_b, aux1, aux2):
    """
    Perform an inplace modular addition of a quantum register 'wires_b' to another quantum register 'wires_a'
    modulo 'N' with auxiliary wires 'aux1' and 'aux2'.

    :param N: The modulus.
    :param wires_a: Quantum wires for the first input.
    :param wires_b: Quantum wires for the second input.
    :param aux1: Auxiliary quantum wires.
    :param aux2: Auxiliary quantum wires.
    """
    new_wires_b = aux1 + wires_b

    # Apply the quantum Fourier transform to 'wires_b'
    qml.QFT(wires=new_wires_b)

    for i in range(len(wires_a)):
        value = 2**(len(wires_a) - 1 - i) % N

        # Controlled modular addition of 'value' to 'wires_b'
        qml.ctrl(add_in_k_N, control=wires_a[i])(value, N, new_wires_b, aux2, False)

    # Inverse quantum Fourier transform on 'wires_b'
    qml.adjoint(qml.QFT)(wires=new_wires_b)

# Create the device
nWireCount = 10
dev = qml.device("default.qubit", wires=nWireCount, shots=1)

@qml.qnode(dev)
def circuit_add_in_N(a, b, N, wires_a, wires_b, aux1, aux2):
    qml.BasisEmbedding(a, wires=wires_a)
    qml.BasisEmbedding(b, wires=wires_b)
    add_in_N(N, wires_a, wires_b, aux1, aux2)
    return qml.sample()

# Set wires
wires_a = list(range(0, 3))
wires_b = list(range(3, 6))
aux1 = [6]
aux2 = [7]

# Example
N = 5  # Input value between [0, 7]
a = 1  # Input value between [0, N-1]
b = 3  # Input value between [0, N-1]
vOutput = circuit_add_in_N(a, b, N, wires_a, wires_b, aux1, aux2)
print(a, "+", b, "mod", N, "=", binary_to_integer(vOutput[wires_b]))


1 + 3 mod 5 = 4


## 4. *Outplace* Quantum-Quantum Adder
Similar to the previous operator but in an outplace fashion.

$$Add_{\text{out}}(N)\ket{a}\ket{b}\ket{0} \rightarrow \ket{a}\ket{b}\ket{a+b \mod N}$$
![outplace_q_c_adder](https://github.com/pifparfait/Efficient-Quantum-Modular-Arithmetics/blob/main/img/add_q_q_out.png?raw=true)

In [7]:
def add_out_N(N, wires_a, wires_b, wires_res, aux1, aux2):
    """
    Perform an outplace modular addition of a quantum register 'wires_b' to another quantum register 'wires_a'
    modulo 'N' with the result stored in 'wires_res', using auxiliary wires 'aux1' and 'aux2'.

    :param N: The modulus.
    :param wires_a: Quantum wires for the first input.
    :param wires_b: Quantum wires for the second input.
    :param wires_res: Quantum wires to store the result.
    :param aux1: Auxiliary quantum wires.
    :param aux2: Auxiliary quantum wires.
    """
    # Set 'b' in 'wires_res'
    for i in range(len(wires_b)):
        qml.CNOT(wires=[wires_b[i], wires_res[i]])

    # Sum 'a' to 'wires_res' using the 'add_in_N' function
    add_in_N(N, wires_a, wires_res, aux1, aux2)

# Create the device
nWireCount = 11
dev = qml.device("default.qubit", wires=nWireCount, shots=1)

@qml.qnode(dev)
def circuit_add_out_N(a, b, N, wires_a, wires_b, wires_res, aux1, aux2):
    qml.BasisEmbedding(a, wires=wires_a)
    qml.BasisEmbedding(b, wires=wires_b)
    add_out_N(N, wires_a, wires_b, wires_res, aux1, aux2)
    return qml.sample()

# Set wires
wires_a = list(range(0, 3))
wires_b = list(range(3, 6))
wires_res = list(range(6, 9))
aux1 = [9]
aux2 = [10]

# Example
N = 5  # Input value between [0, 7]
a = 1  # Input value between [0, N-1]
b = 3  # Input value between [0, N-1]

vOutput = circuit_add_out_N(a, b, N, wires_a, wires_b, wires_res, aux1, aux2)
print(a, "+", b, "mod", N, "=", binary_to_integer(vOutput[wires_res]))


1 + 3 mod 5 = 4


## 5. *Outplace* Quantum-Classic Multiplier
This operator multiplies a quantum register by a classical constant modulo N.
$$Mult_{\text{out}}(k, N)\ket{a}\ket{b} \rightarrow \ket{a}\ket{b + ka \mod N}$$

![outplace_q_c_multplier](https://github.com/pifparfait/Efficient-Quantum-Modular-Arithmetics/blob/main/img/mult_q_c_out.png?raw=true)

In [8]:
def mult_out_k_N(k, N, wires_a, wires_b, wires_aux):
    """
    Perform an outplace modular multiplication of a quantum register 'wires_a' by a classical constant 'k',
    modulo 'N', with the result stored in 'wires_b', using auxiliary wires 'wires_aux'.

    :param k: The classical constant to multiply by.
    :param N: The modulus.
    :param wires_a: Quantum wires for the first input.
    :param wires_b: Quantum wires to store the result.
    :param wires_aux: Auxiliary quantum wires.
    """
    if len(wires_aux) < 2:
        print("Not enough ancillary qubits")
        return

    # Apply Fourier basis
    new_wires_b = [wires_aux[0]] + wires_b

    qml.QFT(wires=new_wires_b)
    
    for i in range(len(wires_a)):
        value = (k * 2**(len(wires_a) - 1 - i)) % N
        qml.ctrl(add_in_k_N, wires_a[i])(value, N, new_wires_b, [wires_aux[1]], False)

    qml.adjoint(qml.QFT)(wires=new_wires_b)

## Create the device
nWireCount = 8
dev = qml.device("default.qubit", wires=nWireCount, shots=1)

@qml.qnode(dev)
def circuit_mult_out_k_N(a, b, k, N, wires_a, wires_b, wires_aux):
    qml.BasisEmbedding(a, wires=wires_a)
    qml.BasisEmbedding(b, wires=wires_b)
    mult_out_k_N(k, N, wires_a, wires_b, wires_aux)
    return qml.sample()

## Set wires
wires_a = list(range(0, 3))
wires_b = list(range(3, 6))
wires_aux = [6, 7]

# Example
N = 6  # Input value between [0, 7]
a = 2  # Input value between [0, N-1]
k = 4  # Input value between [0, N-1]
b = 3  # Input value between [0, N-1]

vOutput = circuit_mult_out_k_N(a, b, k, N, wires_a, wires_b, wires_aux)
print(a, "*", k, "+", b, "mod", N, "=", binary_to_integer(vOutput[wires_b]))


2 * 4 + 3 mod 6 = 5


## 6. *Inplace* Quantum-Classic Multiplier
Similar to the previous operator but in an inplace fashion.
 $$Mult_{\text{in}}(k, N)\ket{a} \rightarrow \ket{ka \mod N}$$ 

To calculate this operator, we will perform the following operations:

\begin{align}
    \ket{a}\ket{0} & \xrightarrow{Mult_{\text{out}}(k,N)} \ket{a}\ket{ka} \\
    & \xrightarrow{\text{SWAP}} \ket{ka}\ket{a} \\
    & \xrightarrow{{Mult_{\text{out}}(k^{-1}, N)}^{\dagger}} \ket{ka}\ket{0}.
    \label{eq:quantum_classic_inplace_mult}
\end{align}



These operations represent the steps involved in calculating the operator.
![inplace_q_c_mult](https://github.com/pifparfait/Efficient-Quantum-Modular-Arithmetics/blob/main/img/mult_q_c_in.png?raw=true)

In [9]:
def mult_in_k_N(k, N, wires_a, wires_support, wires_aux):
    """
    Perform an inplace modular multiplication of a quantum register 'wires_a' by a classical constant 'k',
    modulo 'N', with the result stored in 'wires_a'. Uses auxiliary wires 'wires_support' and 'wires_aux'.

    :param k: The classical constant to multiply by.
    :param N: The modulus.
    :param wires_a: Quantum wires for the input and to store the result.
    :param wires_support: Quantum wires for support.
    :param wires_aux: Auxiliary quantum wires.
    """
    if len(wires_aux) < 2 or not (len(wires_a) == len(wires_support)):
        print("Not enough ancillary qubits")
        return

    if pow(k, N-2, N)*k % N != 1:
        print("FALLO: No existe el inverso de", k, "mod", N)
        return
    
    # Step 1
    mult_out_k_N(k, N, wires_a, wires_support, wires_aux)

    # Step 2
    for i in range(len(wires_a)):
        qml.SWAP(wires=[wires_a[i], wires_support[i]])

    # Step 3
    inv_k = pow(k, N-2, N)  # Calculate the modular inverse
    mult_out_k_N(N - inv_k, N, wires_a, wires_support, wires_aux)

## Create the device
nWireCount = 8
dev = qml.device("default.qubit", wires=nWireCount, shots=1)

@qml.qnode(dev)
def circuit_mult_in_k_N(a, k, N, wires_a, wires_support, wires_aux):
    qml.BasisEmbedding(a, wires=wires_a)
    mult_in_k_N(k, N, wires_a, wires_support, wires_aux)
    return qml.sample()

## Set wires
wires_a = list(range(0, 3))
wires_support = list(range(3, 6))
wires_aux = [6, 7]

# Example
N = 5  # Input value between [0, 7]
a = 1  # Input value between [0, N-1]
k = 3  # Input value between [0, N-1]
vOutput = circuit_mult_in_k_N(a, k, N, wires_a, wires_support, wires_aux)
print(a, "*", k, "mod", N, "=", binary_to_integer(vOutput[wires_a]))


1 * 3 mod 5 = 3


## 7. *Outplace* Quantum-Quantum Multiplier 
This operator multiplies two quantum registers modulo N.
$$Mult_{\text{out}}(N)\ket{a}\ket{b}\ket{0} \rightarrow \ket{a}\ket{b}\ket{ab \mod N}$$ 
![outplace_q_q_mult](https://github.com/pifparfait/Efficient-Quantum-Modular-Arithmetics/blob/main/img/mult_q_q_out.png?raw=true)

In [10]:
def mult_out_N(N, wires_a, wires_b, wires_res, wires_aux):
    """
    Perform an outplace modular multiplication of quantum registers 'wires_a' and 'wires_b',
    with the result stored in 'wires_res'. Uses auxiliary wires 'wires_aux'.

    :param N: The modulus.
    :param wires_a: Quantum wires for the first input.
    :param wires_b: Quantum wires for the second input.
    :param wires_res: Quantum wires to store the result.
    :param wires_aux: Auxiliary quantum wires.
    """
    # Join the auxiliary wires to the result wires
    new_wires_res = [wires_aux[0]] + wires_res
    
    # Set the Fourier basis
    qml.QFT(wires=new_wires_res)
    
    for i in range(len(wires_a)):
        new_i = len(wires_a) - i - 1
        for j in range(len(wires_b)):
            new_j = len(wires_b) - j - 1
            value = 2**(new_i + new_j) % N
            qml.ctrl(add_in_k_N, control=[wires_a[i], wires_b[j]])(value, N, new_wires_res, [wires_aux[1]], False)
    
    # Recover the basis
    qml.adjoint(qml.QFT)(wires=new_wires_res)

## Create the device
nWireCount = 11
dev = qml.device("default.qubit", wires=nWireCount, shots=1)

@qml.qnode(dev)
def circuit_mult_out_N(a, b, N, wires_a, wires_b, wires_res, wires_aux):
    qml.BasisEmbedding(a, wires=wires_a)
    qml.BasisEmbedding(b, wires=wires_b)
    mult_out_N(N, wires_a, wires_b, wires_res, wires_aux)
    return qml.sample()

## Set wires
wires_a = list(range(0, 3))
wires_b = list(range(3, 6))
wires_res = list(range(6, 9))
wires_aux = [9, 10]

# Example
N = 7  # Input value between [0, 7]
a = 3  # Input value between [0, N-1]
b = 5  # Input value between [0, N-1]
vOutput = circuit_mult_out_N(a, b, N, wires_a, wires_b, wires_res, wires_aux)
print(a, "*", b, "mod", N, "=", binary_to_integer(vOutput[wires_res]))


3 * 5 mod 7 = 1


# 8. Modular Exponential Operator
This operator efficiently computes the modular exponentiation of a quantum register.

$$Exp(a, N)\ket{x}\ket{1}\ket{0} \rightarrow \ket{x}\ket{a^x \mod N}\ket{0}$$
![out_place_exp](https://github.com/pifparfait/Efficient-Quantum-Modular-Arithmetics/blob/main/img/exp_operator.png?raw=true)


In [11]:
def Exp_a_N(a, N, wires_x, wires_res, wires_support, wires_aux):
    """
    Exponential operator for quantum registers 'wires_x' with the result stored in 'wires_res'.
    
    :param a: The base of the exponential.
    :param N: The modulus.
    :param wires_x: Quantum wires for the input.
    :param wires_res: Quantum wires to store the result.
    :param wires_support: Quantum wires for intermediate calculations.
    :param wires_aux: Auxiliary quantum wires.
    """
    # Set wires_res -> |1>
    qml.PauliX(wires=wires_res[len(wires_res) - 1])
    
    # Apply the Mult_in
    n = len(wires_x)
    for i in range(n):
        value = a**(2**(n-i-1))
        qml.ctrl(mult_in_k_N, control=[wires_x[i]])(value, N, wires_res, wires_support, wires_aux)

## Create the device
nWireCount = 11
dev = qml.device("default.qubit", wires=nWireCount, shots=1)

@qml.qnode(dev)
def circuit_exp_a_N(x, a, N, wires_x, wires_res, wires_support, wires_aux):
    qml.BasisEmbedding(x, wires=wires_x)
    Exp_a_N(a, N, wires_x, wires_res, wires_support, wires_aux)
    return qml.sample()

## Set wires
wires_x = list(range(0, 3))
wires_res = list(range(3, 6))
wires_support = list(range(6, 9))
wires_aux = [9, 10]

# Example
N = 7  # Input value between [0, 7]
x = 3  # Input value between [0, N-1]
a = 2  # Input value between [0, N-1]
vOutput = circuit_exp_a_N(x, a, N, wires_x, wires_res, wires_support, wires_aux)
print(a, "**", x, "mod", N, "=", binary_to_integer(vOutput[wires_res]))
print("The expected result is", a**x % N)


2 ** 3 mod 7 = 1
The expected result is 1


## Period finding

This cell introduces the concept of using the **exponential operator** in Shor's quantum algorithm for integer factorization. It encourages users to explore how this operator can help find the period, a crucial step in Shor's algorithm, by referring to an included quantum circuit diagram. The image provides a visual representation of part of the algorithm. It is a starting point for users interested in implementing Shor's algorithm for integer factorization using quantum computing techniques.

![period_finding](https://github.com/pifparfait/Efficient-Quantum-Modular-Arithmetics/blob/main/img/p_finding.png?raw=true)


In [39]:
## Create the quantum device
nWireCount = 11
dev = qml.device("default.qubit", wires=nWireCount)

@qml.qnode(dev)
def circuit_exp_a_N(a, N, wires_x, wires_res, wires_support, wires_aux):
    # Apply Hadamard gates to all wires in wires_x
    for i in range(len(wires_x)):
        qml.Hadamard(wires=i)

    # Apply the Exponential Operator
    Exp_a_N(a, N, wires_x, wires_res, wires_support, wires_aux)

    # Apply the inverse Quantum Fourier Transform
    qml.adjoint(qml.QFT(wires=wires_x))

    return qml.probs(wires=wires_x)

## Set wire configurations
wires_x = list(range(0, 3))
wires_res = list(range(3, 6))
wires_support = list(range(6, 9))
wires_aux = [9, 10]

# Example
N = 7  # Input value between [0, 7]
a = 3  # Input value between [0, N-1]
vOutput = circuit_exp_a_N(a, N, wires_x, wires_res, wires_support, wires_aux)
vOutput

tensor([0.1875, 0.125 , 0.0625, 0.125 , 0.1875, 0.125 , 0.0625, 0.125 ], requires_grad=True)

## How to cite this work:


@misc{atchadeadelomou2023efficient,
      title={Efficient Quantum Modular Arithmetics for the ISQ Era}, 
      author={Parfait Atchade-Adelomou and Saul Gonzalez},
      year={2023},
      eprint={2311.08555},
      archivePrefix={arXiv},
      primaryClass={quant-ph}
}